<a href="https://colab.research.google.com/github/alphym/NLP-fellowship/blob/main/igihe_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment
Based on the above, get the main articles from igihe from February 2022 - present

Steps to do this


1.   Get the links to the main pages from january. Create a list
2.   In each link, get all the links to the main articles
3.   For each article, get the main tag that holds the texts
4.   Get the text and store them in a txt file. The data will be used in week 2
5.   Each article its own txt file. Naming is the date_article_1


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
snap = []
err = []
for month in range(1,11):
    for day in range(1,32):
        link = requests.get(url='http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}'.format(month, day))
        try:
            if link.json()['archived_snapshots']['closest']['available']:
              #we store each snapshot link into snap list
                snap.append(link.json()['archived_snapshots']['closest']['url'])
        except KeyError:
            err.append(link.json())
            

In [35]:
len(snap)

305

===== downside we are going to remove the duplicate in all snappshoot captured

In [36]:
snap_set  = set(snap)

In [ ]:
snap_r = list(snap_set)
len(snap_r)

==== here we are going to BeautifulSoup to get all content in all snapshoot

In [ ]:
#Fetch all the titles and their link from wayback archivr

links_art = []

for lin in snap_r:
  content = requests.get(lin).content #pass each snapshot link to the requests
  soup =  BeautifulSoup(content, "html.parser") #pass each snapshot content html parser
  articles = soup.find_all('span', class_="homenews-title") #look for all title in the snapshot
  links_art.append(articles)

In [ ]:
len(list(snap_r))

In [ ]:
#Actual link
snapshot_link = []
title_ = []

#Loop all the 184 snapshots
for index, snapshot in enumerate(list(snap_r)):
  #In each title assign it's snapshop prefix link
  for title in links_art[index]:
    snapshot_link.append(snapshot)
    title_.append(str(title.find('a')['href']))

In [ ]:
len(snapshot_link)

In [ ]:
df = pd.DataFrame(list(zip(snapshot_link, title_)), columns=['Prefix', 'Title'])

In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
df.drop_duplicates(['Title'])

In [ ]:
link = df['Prefix'][0] + df['Title'][0]

content = requests.get(link).content

raw_content = BeautifulSoup(content, 'html.parser')

In [ ]:
body = raw_content.find_all('div', class_="fulltext margintop10") 

text = ''
par_ = BeautifulSoup(str(body)).find_all('p')

for line in par_:
  text = text + '\n' + line.get_text()



In [ ]:
def get_text(link):
  content = requests.get(link).content
  raw_content = BeautifulSoup(content, 'html.parser')

  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()

In [ ]:
def get_article_text(link):
  content = requests.get(link).content
  body = BeautifulSoup(content, 'html.parser').find_all('div', class_='fulltext margintop10')

  text = ''
  try:
    par_ = BeautifulSoup(str(body)).find_all('p')


    for line in par_:
      text = text + '\n' + line.get_text()
  except AttributeError:
    pass
  return text

In [31]:
articles = []

for index, article in enumerate(df['Title']):
  link =df['Prefix'][index]+article
  txt = get_article_text(link)
  articles.append(txt)

KeyboardInterrupt: ignored

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_all_text_file(index):
  link = df['Prefix'][index] + df['Title'][index]

  content = requests.get(link).content

  raw_content = BeautifulSoup(content, 'html.parser')
  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()
  return text


In [ ]:
for index in range(len(df)):
  new_t = get_all_text_file(index)
  y = open('/content/drive/MyDrive/NLP-FELLOWSHIP/WEEK1-NLP/content.txt', 'w+',  encoding='utf-8')
  y.write(new_t+' \n')